In [1]:
import os
import sys
import math
import pandas as pd
import json
file_path1 = '/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/build'
sys.path.append(file_path1)
print(file_path1)
import pybind_dlp
from map_test import *
from bokeh.plotting import figure, output_notebook, show, output_file
output_notebook()
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool, ColumnDataSource, CrosshairTool, CustomJS, Div, TapTool
from bokeh.layouts import column
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, Div, LabelSet, Button
from bokeh.layouts import column
from bokeh.models.widgets import Toggle


/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/build


Loading BokehJS ...

In [2]:
root_dir = '/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math'
athena_dir = os.path.join(root_dir, 'athena')
athena_dir
data_raw_path = os.path.join(athena_dir,'data_demo','raw')
sub_dirs = [d for d in os.listdir(data_raw_path) if os.path.isdir(os.path.join(data_raw_path, d))]
sub_dirs
map_json_path = os.path.join(data_raw_path,sub_dirs[0],f'log_map_archive_{sub_dirs[0]}.json')
map_json_path

'/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/data_demo/raw/000a07a3-fec1-47d5-8ba7-e8610409efb3/log_map_archive_000a07a3-fec1-47d5-8ba7-e8610409efb3.json'

In [3]:
def plot_index_list(data, color, name):
    x = list(range(len(data)))
    y = data
    source = ColumnDataSource(data=dict(x=x, y=y, values=data))
    # 创建图形对象
    p = figure(tools="pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select", 
             title=name, width=400, height=300)
    # 添加散点图
    p.scatter('x', 'y', size=5, source=source, color = color)

    # 添加 HoverTool 工具
    hover = HoverTool()
    hover.tooltips = [("Index", "$index"), ("Value", "@values")]
    p.add_tools(hover)

    # 设置图形的标签
    p.xaxis.axis_label = 'Index'
    p.yaxis.axis_label = 'Value'

    return p

def construct_source_data2(x_list, y_list):
    # 创建一个 ColumnDataSource 对象
    return ColumnDataSource(data=dict(x=x_list, y=y_list, index=list(range(len(x_list)))))

def plot_two_position_data(pl_x_list, pl_y_list, pt_x_list, pt_y_list):
    # 创建数据源
    pl_source = construct_source_data2(pl_x_list, pl_y_list) 
    pt_source = construct_source_data2(pt_x_list, pt_y_list) 
    
    # 创建图形对象
    plot = figure(
        title="point_position",
        x_axis_label="X Coordinate",
        y_axis_label="Y Coordinate",
        width=750,
        height=900,
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
    )

    # 添加散点图
    pl_glyph = plot.scatter("x", "y", size=10, color="red", 
                            alpha=0.5, legend_label="polygon_position", source=pl_source)
    point_glyph = plot.scatter("x", "y", size=10, color="blue", 
                               alpha=0.5, legend_label="point_position", source=pt_source)

    # 添加 HoverTool 工具
    hoverpl = HoverTool(renderers=[pl_glyph], tooltips=[("Index", "$index"), ("(X,Y)", "(@x, @y)")])
    hoverpt = HoverTool(renderers=[point_glyph], tooltips=[("Index", "$index"), ("(X,Y)", "(@x, @y)")])
    plot.add_tools(hoverpl, hoverpt)

    # 添加 LabelSet
    pl_labels = LabelSet(x='x', y='y', text='index', level='glyph',
                         source=pl_source, x_offset=10, y_offset=10, 
                         text_font_size='10pt', text_color='red', visible=True)
    pt_labels = LabelSet(x='x', y='y', text='index', level='glyph',
                         source=pt_source, x_offset=10, y_offset=10, 
                         text_font_size='10pt', text_color='blue', visible=True)

    plot.add_layout(pl_labels)
    plot.add_layout(pt_labels)

    # 添加 CrosshairTool
    from bokeh.models import CrosshairTool
    crosshair = CrosshairTool()
    plot.add_tools(crosshair)

    # 创建一个 Div 元素来显示坐标
    div = Div(width=200, height=50)

    # 创建 CustomJS 回调来更新 Div 内容
    callback = CustomJS(args=dict(div=div), code="""
        const {x, y} = cb_obj;
        div.text = `x: ${x.toFixed(7)}, y: ${y.toFixed(7)}`;
    """)
    plot.js_on_event('mousemove', callback)

    # 创建 Toggle 控件来控制 LabelSet 的显示和隐藏
    toggle = Toggle(label="Toggle Labels", button_type="success", width=150)

    toggle_callback = CustomJS(args=dict(pl_labels=pl_labels, pt_labels=pt_labels), code="""
        pl_labels.visible = !pl_labels.visible;
        pt_labels.visible = !pt_labels.visible;
    """)
    toggle.js_on_click(toggle_callback)

    # 设置图例点击策略
    plot.legend.click_policy = "hide"

    # 布局
    layout = column(plot, toggle, div)
    
    return layout



In [5]:
json_data = load_json_file(map_json_path)
ans = call_pybind_dlp(json_data)
# return map_pl_po, map_pl_or, map_pl_type, map_pl_isin, \
#         map_pt_po, map_pt_or, map_pt_mag, map_pt_type, map_pt_side

# plot polygon
pl_x_list = []
pl_y_list = []
for pt in ans[0]:
    pl_x_list.append(pt[0])
    pl_y_list.append(pt[1])
    
pl_or_list = []
for pt in ans[1]:
    pl_or_list.append(pt)
plot_pl_or = plot_index_list(pl_or_list, 'black', "map_polygon_orientation")

pl_type_list = []
for pt in ans[2]:
    pl_type_list.append(pt)
plot_pl_type = plot_index_list(pl_type_list, 'black', "map_polygon_type")
    
pl_isin_list = []
for pt in ans[3]:
    pl_isin_list.append(pt)   
plot_pl_isin = plot_index_list(pl_isin_list, 'black', "map_polygon_is_intersection")

# plot point
pt_x_list = []
pt_y_list = []
for pt in ans[4]:
    pt_x_list.append(pt[0])
    pt_y_list.append(pt[1])
    
pt_or_list = []
for pt in ans[5]:
    pt_or_list.append(pt)
plot_pt_or = plot_index_list(pt_or_list, 'red', "map_point_orientation")

pl_mag_list = []
for pt in ans[6]:
    pl_mag_list.append(pt)
plot_pt_mag = plot_index_list(pl_mag_list, 'red', "map_point_magnitude")

pl_type_list = []
for pt in ans[7]:
    pl_type_list.append(pt) 
plot_pt_type = plot_index_list(pl_type_list, 'red', "map_point_type")
  
pl_side_list = []
for pt in ans[8]:
    pl_side_list.append(pt) 
plot_pt_side = plot_index_list(pl_side_list, 'red', "map_point_side")

position_plot = plot_two_position_data(pl_x_list, pl_y_list, pt_x_list, pt_y_list)
row1 = row(plot_pl_or, plot_pl_type, plot_pl_isin)
row2 = column(row(plot_pt_or, plot_pt_mag), 
              row(plot_pt_type, plot_pt_side))

show(row(position_plot, column(row1, row2)))


run here is
